<a href="https://colab.research.google.com/github/kaindoh/Sendy-zindi-project/blob/master/sendy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold, GridSearchCV, RandomizedSearchCV,cross_val_score
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, GradientBoostingRegressor,ExtraTreesRegressor
from xgboost import XGBRFRegressor
import lightgbm as lgb
import xgboost as xgb
import requests
from io import StringIO
import hyperopt
import datetime
import datetime as dt
import warnings
warnings.filterwarnings('ignore')

In [0]:
train = 'https://drive.google.com/file/d/1ZP9pFBATu38l97Tut5hKLvpzKRLFXX_P/view?usp=sharing'
test = 'https://drive.google.com/file/d/1_aElMoEIRs55avOafA7U1_YXEuaDBXLh/view?usp=sharing'
submission = 'https://drive.google.com/file/d/1mqXS8euMqF9_bhTEU6O9cLHoX2FI_5HD/view?usp=sharing'
dictionary = 'https://drive.google.com/file/d/1juqltwSs6OXQgJJEhTxD7Gm443fnLpCp/view?usp=sharing'
riders = 'https://drive.google.com/file/d/19-aVgAcKRxX_Tk9StUQMNeAUVi0ZTo9K/view?usp=sharing'

def read_csv(url):
  url = 'https://drive.google.com/uc?export=download&id=' + url.split('/')[-2]
  csv_raw = requests.get(url).text
  csv = StringIO(csv_raw)
  return csv

df = pd.read_csv(read_csv(train))
df1 = pd.read_csv(read_csv(test))
sub = pd.read_csv(read_csv(submission))
dictionary = pd.read_csv(read_csv(dictionary))
riders = pd.read_csv(read_csv(riders))

### Cleaning column names

In [287]:
df.columns

Index(['Order No', 'User Id', 'Vehicle Type', 'Platform Type',
       'Personal or Business', 'Placement - Day of Month',
       'Placement - Weekday (Mo = 1)', 'Placement - Time',
       'Confirmation - Day of Month', 'Confirmation - Weekday (Mo = 1)',
       'Confirmation - Time', 'Arrival at Pickup - Day of Month',
       'Arrival at Pickup - Weekday (Mo = 1)', 'Arrival at Pickup - Time',
       'Pickup - Day of Month', 'Pickup - Weekday (Mo = 1)', 'Pickup - Time',
       'Arrival at Destination - Day of Month',
       'Arrival at Destination - Weekday (Mo = 1)',
       'Arrival at Destination - Time', 'Distance (KM)', 'Temperature',
       'Precipitation in millimeters', 'Pickup Lat', 'Pickup Long',
       'Destination Lat', 'Destination Long', 'Rider Id',
       'Time from Pickup to Arrival'],
      dtype='object')

In [288]:

df.columns = df.columns.str.lower().str.replace(' ', '_').str.replace('-', '_').str.replace('=', '_')
df.columns = df.columns.str.replace('__', '_')
df.columns = df.columns.str.replace('(', '').str.replace(')', '')
df.columns = df.columns.str.replace('__', '_')
df.columns

Index(['order_no', 'user_id', 'vehicle_type', 'platform_type',
       'personal_or_business', 'placement_day_of_month',
       'placement_weekday_mo_1', 'placement_time', 'confirmation_day_of_month',
       'confirmation_weekday_mo_1', 'confirmation_time',
       'arrival_at_pickup_day_of_month', 'arrival_at_pickup_weekday_mo_1',
       'arrival_at_pickup_time', 'pickup_day_of_month', 'pickup_weekday_mo_1',
       'pickup_time', 'arrival_at_destination_day_of_month',
       'arrival_at_destination_weekday_mo_1', 'arrival_at_destination_time',
       'distance_km', 'temperature', 'precipitation_in_millimeters',
       'pickup_lat', 'pickup_long', 'destination_lat', 'destination_long',
       'rider_id', 'time_from_pickup_to_arrival'],
      dtype='object')

In [0]:
# Arrival at Destination - Day of Month
# Arrival at Destination - Weekday (Mo = 1)
# Arrival at Destination - Time
# Time from Pickup to Arrival

In [0]:
df.drop(['arrival_at_destination_day_of_month', 'arrival_at_destination_weekday_mo_1','arrival_at_destination_time'],axis=1,inplace=True)

In [291]:

df1.columns = df1.columns.str.lower().str.replace(' ', '_').str.replace('-', '_').str.replace('=', '_')
df1.columns = df1.columns.str.replace('__', '_')
df1.columns = df1.columns.str.replace('(', '').str.replace(')', '')
df1.columns = df1.columns.str.replace('__', '_')
df1.columns

Index(['order_no', 'user_id', 'vehicle_type', 'platform_type',
       'personal_or_business', 'placement_day_of_month',
       'placement_weekday_mo_1', 'placement_time', 'confirmation_day_of_month',
       'confirmation_weekday_mo_1', 'confirmation_time',
       'arrival_at_pickup_day_of_month', 'arrival_at_pickup_weekday_mo_1',
       'arrival_at_pickup_time', 'pickup_day_of_month', 'pickup_weekday_mo_1',
       'pickup_time', 'distance_km', 'temperature',
       'precipitation_in_millimeters', 'pickup_lat', 'pickup_long',
       'destination_lat', 'destination_long', 'rider_id'],
      dtype='object')

### Ordering the dataset

In [292]:
df.columns


Index(['order_no', 'user_id', 'vehicle_type', 'platform_type',
       'personal_or_business', 'placement_day_of_month',
       'placement_weekday_mo_1', 'placement_time', 'confirmation_day_of_month',
       'confirmation_weekday_mo_1', 'confirmation_time',
       'arrival_at_pickup_day_of_month', 'arrival_at_pickup_weekday_mo_1',
       'arrival_at_pickup_time', 'pickup_day_of_month', 'pickup_weekday_mo_1',
       'pickup_time', 'distance_km', 'temperature',
       'precipitation_in_millimeters', 'pickup_lat', 'pickup_long',
       'destination_lat', 'destination_long', 'rider_id',
       'time_from_pickup_to_arrival'],
      dtype='object')

In [0]:
# #Stripping the order no to have whole numbers
# df['order_no']= df['order_no'].str.replace('Order_No_', '')

# #Changing the data types
# df['order_no']=df['order_no'].astype(int)
# df.sort_values(by=['order_no'], ascending= True)


In [0]:
# df1['order_no']= df1['order_no'].str.replace('Order_No_', '')

# #Changing the data types
# df1['order_no']=df1['order_no'].astype(int)
# df1.sort_values(by=['order_no'], ascending= True)


In [0]:
df['personal_or_business']=df['personal_or_business'].astype("category")
df1['personal_or_business']=df1['personal_or_business'].astype("category")

### Filling Temperature missing values

In [0]:
df['temperature'].fillna(df['temperature'].mean(), inplace= True)

In [0]:
df1['temperature'].fillna(df['temperature'].mean(), inplace= True)

In [0]:
# #Forward Fill Temprature Column
# df1['temperature'].fillna(method='ffill', inplace= True)

# #Backward Fill Temprature column
# df1['temperature'].fillna(method='bfill', inplace= True)
#Filling in the Precipitation column with 0
df['precipitation_in_millimeters'].fillna(0, inplace= True)

#Filling in the Precipitation column with 0
df1['precipitation_in_millimeters'].fillna(0, inplace= True)

In [297]:
df.head()

,order_no,user_id,vehicle_type,platform_type,personal_or_business,placement_day_of_month,placement_weekday_mo_1,placement_time,confirmation_day_of_month,confirmation_weekday_mo_1,confirmation_time,arrival_at_pickup_day_of_month,arrival_at_pickup_weekday_mo_1,arrival_at_pickup_time,pickup_day_of_month,pickup_weekday_mo_1,pickup_time,distance_km,temperature,precipitation_in_millimeters,pickup_lat,pickup_long,destination_lat,destination_long,rider_id,time_from_pickup_to_arrival
0,Order_No_4211,User_Id_633,Bike,3,Business,9,5,9:35:46 AM,9,5,9:40:10 AM,9,5,10:04:47 AM,9,5,10:27:30 AM,4,20.400000,0.0,-1.317755,36.830370,-1.300406,36.829741,Rider_Id_432,745
1,Order_No_25375,User_Id_2285,Bike,3,Personal,12,5,11:16:16 AM,12,5,11:23:21 AM,12,5,11:40:22 AM,12,5,11:44:09 AM,16,26.400000,0.0,-1.351453,36.899315,-1.295004,36.814358,Rider_Id_856,1993
2,Order_No_1899,User_Id_265,Bike,3,Business,30,2,12:39:25 PM,30,2,12:42:44 PM,30,2,12:49:34 PM,30,2,12:53:03 PM,3,23.258889,0.0,-1.308284,36.843419,-1.300921,36.828195,Rider_Id_155,455
3,Order_No_9336,User_Id_1402,Bike,3,Business,15,5,9:25:34 AM,15,5,9:26:05 AM,15,5,9:37:56 AM,15,5,9:43:06 AM,9,19.200000,0.0,-1.281301,36.832396,-1.257147,36.795063,Rider_Id_855,1341
4,Order_No_27883,User_Id_1737,Bike,1,Personal,13,1,9:55:18 AM,13,1,9:56:18 AM,13,1,10:03:53 AM,13,1,10:05:23 AM,9,15.400000,0.0,-1.266597,36.792118,-1.295041,36.809817,Rider_Id_770,1214


In [298]:
df1.head()

,order_no,user_id,vehicle_type,platform_type,personal_or_business,placement_day_of_month,placement_weekday_mo_1,placement_time,confirmation_day_of_month,confirmation_weekday_mo_1,confirmation_time,arrival_at_pickup_day_of_month,arrival_at_pickup_weekday_mo_1,arrival_at_pickup_time,pickup_day_of_month,pickup_weekday_mo_1,pickup_time,distance_km,temperature,precipitation_in_millimeters,pickup_lat,pickup_long,destination_lat,destination_long,rider_id
0,Order_No_19248,User_Id_3355,Bike,3,Business,27,3,4:44:10 PM,27,3,4:44:29 PM,27,3,4:53:04 PM,27,3,5:06:47 PM,8,23.258889,0.0,-1.333275,36.870815,-1.305249,36.822390,Rider_Id_192
1,Order_No_12736,User_Id_3647,Bike,3,Business,17,5,12:57:35 PM,17,5,12:59:17 PM,17,5,1:20:27 PM,17,5,1:25:37 PM,5,23.258889,0.0,-1.272639,36.794723,-1.277007,36.823907,Rider_Id_868
2,Order_No_768,User_Id_2154,Bike,3,Business,27,4,11:08:14 AM,27,4,11:25:05 AM,27,4,11:33:20 AM,27,4,11:57:54 AM,5,22.800000,0.0,-1.290894,36.822971,-1.276574,36.851365,Rider_Id_26
3,Order_No_15332,User_Id_2910,Bike,3,Business,17,1,1:51:35 PM,17,1,1:53:27 PM,17,1,2:02:41 PM,17,1,2:16:52 PM,5,24.500000,0.0,-1.290503,36.809646,-1.303382,36.790658,Rider_Id_685
4,Order_No_21373,User_Id_1205,Bike,3,Business,11,2,11:30:28 AM,11,2,11:34:45 AM,11,2,11:47:19 AM,11,2,11:56:04 AM,6,24.400000,0.0,-1.281081,36.814423,-1.266467,36.792161,Rider_Id_858


In [299]:
df.isnull().sum()

order_no                          0
user_id                           0
vehicle_type                      0
platform_type                     0
personal_or_business              0
placement_day_of_month            0
placement_weekday_mo_1            0
placement_time                    0
confirmation_day_of_month         0
confirmation_weekday_mo_1         0
confirmation_time                 0
arrival_at_pickup_day_of_month    0
arrival_at_pickup_weekday_mo_1    0
arrival_at_pickup_time            0
pickup_day_of_month               0
pickup_weekday_mo_1               0
pickup_time                       0
distance_km                       0
temperature                       0
precipitation_in_millimeters      0
pickup_lat                        0
pickup_long                       0
destination_lat                   0
destination_long                  0
rider_id                          0
time_from_pickup_to_arrival       0
dtype: int64

In [300]:
df1.isnull().sum()

order_no                          0
user_id                           0
vehicle_type                      0
platform_type                     0
personal_or_business              0
placement_day_of_month            0
placement_weekday_mo_1            0
placement_time                    0
confirmation_day_of_month         0
confirmation_weekday_mo_1         0
confirmation_time                 0
arrival_at_pickup_day_of_month    0
arrival_at_pickup_weekday_mo_1    0
arrival_at_pickup_time            0
pickup_day_of_month               0
pickup_weekday_mo_1               0
pickup_time                       0
distance_km                       0
temperature                       0
precipitation_in_millimeters      0
pickup_lat                        0
pickup_long                       0
destination_lat                   0
destination_long                  0
rider_id                          0
dtype: int64

### Creating hour and minute variables

In [0]:
time_cols = ['placement_time', 'confirmation_time', 'pickup_time']
for col in time_cols:
   df[col] = pd.to_datetime(df[col])
   df[col.split('_')[0] + '_hour'] = df[col].dt.hour
   df[col.split('_')[0] + '_minute'] = df[col].dt.minute
   df[col] = [time.time() for time in df[col]]

In [0]:
time_cols = ['placement_time', 'confirmation_time', 'pickup_time']
for col in time_cols:
   df1[col] = pd.to_datetime(df1[col])
   df1[col.split('_')[0] + '_hour'] = df1[col].dt.hour
   df1[col.split('_')[0] + '_minute'] = df1[col].dt.minute
   df1[col] = [time.time() for time in df1[col]]

In [0]:
time_col = ["arrival_at_pickup_time"]
for col in time_col:
   df[col] = pd.to_datetime(df[col])
   df[col + '_hour'] = df[col].dt.hour
   df[col + '_minute'] = df[col].dt.minute
   df[col] = [time.time() for time in df[col]]

In [0]:
time_col = ["arrival_at_pickup_time",]
for col in time_col:
   df1[col] = pd.to_datetime(df1[col])
   df1[col + '_hour'] = df1[col].dt.hour
   df1[col + '_minute'] = df1[col].dt.minute
   df1[col] = [time.time() for time in df1[col]]

In [305]:
df.head()

,order_no,user_id,vehicle_type,platform_type,personal_or_business,placement_day_of_month,placement_weekday_mo_1,placement_time,confirmation_day_of_month,confirmation_weekday_mo_1,confirmation_time,arrival_at_pickup_day_of_month,arrival_at_pickup_weekday_mo_1,arrival_at_pickup_time,pickup_day_of_month,pickup_weekday_mo_1,pickup_time,distance_km,temperature,precipitation_in_millimeters,pickup_lat,pickup_long,destination_lat,destination_long,rider_id,time_from_pickup_to_arrival,placement_hour,placement_minute,confirmation_hour,confirmation_minute,pickup_hour,pickup_minute,arrival_at_pickup_time_hour,arrival_at_pickup_time_minute
0,Order_No_4211,User_Id_633,Bike,3,Business,9,5,09:35:46,9,5,09:40:10,9,5,10:04:47,9,5,10:27:30,4,20.400000,0.0,-1.317755,36.830370,-1.300406,36.829741,Rider_Id_432,745,9,35,9,40,10,27,10,4
1,Order_No_25375,User_Id_2285,Bike,3,Personal,12,5,11:16:16,12,5,11:23:21,12,5,11:40:22,12,5,11:44:09,16,26.400000,0.0,-1.351453,36.899315,-1.295004,36.814358,Rider_Id_856,1993,11,16,11,23,11,44,11,40
2,Order_No_1899,User_Id_265,Bike,3,Business,30,2,12:39:25,30,2,12:42:44,30,2,12:49:34,30,2,12:53:03,3,23.258889,0.0,-1.308284,36.843419,-1.300921,36.828195,Rider_Id_155,455,12,39,12,42,12,53,12,49
3,Order_No_9336,User_Id_1402,Bike,3,Business,15,5,09:25:34,15,5,09:26:05,15,5,09:37:56,15,5,09:43:06,9,19.200000,0.0,-1.281301,36.832396,-1.257147,36.795063,Rider_Id_855,1341,9,25,9,26,9,43,9,37
4,Order_No_27883,User_Id_1737,Bike,1,Personal,13,1,09:55:18,13,1,09:56:18,13,1,10:03:53,13,1,10:05:23,9,15.400000,0.0,-1.266597,36.792118,-1.295041,36.809817,Rider_Id_770,1214,9,55,9,56,10,5,10,3


### Creating peak and offpeak variables

In [306]:
df.columns

Index(['order_no', 'user_id', 'vehicle_type', 'platform_type',
       'personal_or_business', 'placement_day_of_month',
       'placement_weekday_mo_1', 'placement_time', 'confirmation_day_of_month',
       'confirmation_weekday_mo_1', 'confirmation_time',
       'arrival_at_pickup_day_of_month', 'arrival_at_pickup_weekday_mo_1',
       'arrival_at_pickup_time', 'pickup_day_of_month', 'pickup_weekday_mo_1',
       'pickup_time', 'distance_km', 'temperature',
       'precipitation_in_millimeters', 'pickup_lat', 'pickup_long',
       'destination_lat', 'destination_long', 'rider_id',
       'time_from_pickup_to_arrival', 'placement_hour', 'placement_minute',
       'confirmation_hour', 'confirmation_minute', 'pickup_hour',
       'pickup_minute', 'arrival_at_pickup_time_hour',
       'arrival_at_pickup_time_minute'],
      dtype='object')

In [0]:
#7-5
hour_cols = ['placement_hour',	'confirmation_hour',	'pickup_hour',"arrival_at_pickup_time_hour"]

for col in hour_cols:
  df[col.split('_')[0] + '_ap'] = df[col].apply(lambda x: 1 if( x >= 7 & x <= 17) else 0)
  

In [0]:
#7-5
hour_cols = ['placement_hour',	'confirmation_hour',	'pickup_hour',"arrival_at_pickup_time_hour"]

for col in hour_cols:
  df1[col.split('_')[0] + '_ap'] = df1[col].apply(lambda x: 1 if( x >= 7 & x <= 17) else 0)
  

### Creating new cluster variables

In [0]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters = 15, init ='k-means++')
kmeans.fit(df[['pickup_lat', 'pickup_long']]) # Compute k-means clustering.
df['pickup_cluster_label'] = kmeans.fit_predict(df[['pickup_lat', 'pickup_long']])
centers1 = kmeans.cluster_centers_ # Coordinates of cluster centers.
labels1 = kmeans.predict(df[['pickup_lat', 'pickup_long']]) # Labels of each point
df['pickup_cluster_label'] = df['pickup_cluster_label'].astype('category')
kmeans = KMeans(n_clusters = 15, init ='k-means++')
kmeans.fit(df[['destination_lat', 'destination_long']]) # Compute k-means clustering.
df['destination_cluster_label'] = kmeans.fit_predict(df[['destination_lat', 'destination_long']])
centers = kmeans.cluster_centers_ # Coordinates of cluster centers.
labels = kmeans.predict(df[['destination_lat', 'destination_long']]) # Labels of each point
df['destination_cluster_label'] =df['destination_cluster_label'].astype('category')


In [0]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters = 15, init ='k-means++')
kmeans.fit(df1[['pickup_lat', 'pickup_long']]) # Compute k-means clustering.
df1['pickup_cluster_label'] = kmeans.fit_predict(df1[['pickup_lat', 'pickup_long']])
centers1 = kmeans.cluster_centers_ # Coordinates of cluster centers.
labels1 = kmeans.predict(df1[['pickup_lat', 'pickup_long']]) # Labels of each point
df1['pickup_cluster_label'] = df1['pickup_cluster_label'].astype('category')
kmeans = KMeans(n_clusters = 15, init ='k-means++')
kmeans.fit(df1[['destination_lat', 'destination_long']]) # Compute k-means clustering.
df1['destination_cluster_label'] = kmeans.fit_predict(df1[['destination_lat', 'destination_long']])
centers = kmeans.cluster_centers_ # Coordinates of cluster centers.
labels = kmeans.predict(df1[['destination_lat', 'destination_long']]) # Labels of each point
df1['destination_cluster_label'] =df1['destination_cluster_label'].astype('category')

In [311]:
df.head()

,order_no,user_id,vehicle_type,platform_type,personal_or_business,placement_day_of_month,placement_weekday_mo_1,placement_time,confirmation_day_of_month,confirmation_weekday_mo_1,confirmation_time,arrival_at_pickup_day_of_month,arrival_at_pickup_weekday_mo_1,arrival_at_pickup_time,pickup_day_of_month,pickup_weekday_mo_1,pickup_time,distance_km,temperature,precipitation_in_millimeters,pickup_lat,pickup_long,destination_lat,destination_long,rider_id,time_from_pickup_to_arrival,placement_hour,placement_minute,confirmation_hour,confirmation_minute,pickup_hour,pickup_minute,arrival_at_pickup_time_hour,arrival_at_pickup_time_minute,placement_ap,confirmation_ap,pickup_ap,arrival_ap,pickup_cluster_label,destination_cluster_label
0,Order_No_4211,User_Id_633,Bike,3,Business,9,5,09:35:46,9,5,09:40:10,9,5,10:04:47,9,5,10:27:30,4,20.400000,0.0,-1.317755,36.830370,-1.300406,36.829741,Rider_Id_432,745,9,35,9,40,10,27,10,4,1,1,1,1,14,6
1,Order_No_25375,User_Id_2285,Bike,3,Personal,12,5,11:16:16,12,5,11:23:21,12,5,11:40:22,12,5,11:44:09,16,26.400000,0.0,-1.351453,36.899315,-1.295004,36.814358,Rider_Id_856,1993,11,16,11,23,11,44,11,40,1,1,1,1,11,6
2,Order_No_1899,User_Id_265,Bike,3,Business,30,2,12:39:25,30,2,12:42:44,30,2,12:49:34,30,2,12:53:03,3,23.258889,0.0,-1.308284,36.843419,-1.300921,36.828195,Rider_Id_155,455,12,39,12,42,12,53,12,49,1,1,1,1,14,6
3,Order_No_9336,User_Id_1402,Bike,3,Business,15,5,09:25:34,15,5,09:26:05,15,5,09:37:56,15,5,09:43:06,9,19.200000,0.0,-1.281301,36.832396,-1.257147,36.795063,Rider_Id_855,1341,9,25,9,26,9,43,9,37,1,1,1,1,2,10
4,Order_No_27883,User_Id_1737,Bike,1,Personal,13,1,09:55:18,13,1,09:56:18,13,1,10:03:53,13,1,10:05:23,9,15.400000,0.0,-1.266597,36.792118,-1.295041,36.809817,Rider_Id_770,1214,9,55,9,56,10,5,10,3,1,1,1,1,9,6


# Cyclic of time

In [0]:
# x = df.drop(['user_id','vehicle_type', 'confirmation_time', 'arrival_at_pickup_time','pickup_time','placement_time','arrival_at_destination_day_of_month', 'arrival_at_destination_weekday_mo_1', 'arrival_at_destination_time','rider_id'],axis=1)
# X = pd.get_dummies(x)
# y = df['time_from_pickup_to_arrival']

# train,test,y_train,y_test = train_test_split(X,y,random_state =0 ,test_size = 0.2)

In [0]:

train = df.copy()
test = df1.copy()

In [314]:
test.head()

,order_no,user_id,vehicle_type,platform_type,personal_or_business,placement_day_of_month,placement_weekday_mo_1,placement_time,confirmation_day_of_month,confirmation_weekday_mo_1,confirmation_time,arrival_at_pickup_day_of_month,arrival_at_pickup_weekday_mo_1,arrival_at_pickup_time,pickup_day_of_month,pickup_weekday_mo_1,pickup_time,distance_km,temperature,precipitation_in_millimeters,pickup_lat,pickup_long,destination_lat,destination_long,rider_id,placement_hour,placement_minute,confirmation_hour,confirmation_minute,pickup_hour,pickup_minute,arrival_at_pickup_time_hour,arrival_at_pickup_time_minute,placement_ap,confirmation_ap,pickup_ap,arrival_ap,pickup_cluster_label,destination_cluster_label
0,Order_No_19248,User_Id_3355,Bike,3,Business,27,3,16:44:10,27,3,16:44:29,27,3,16:53:04,27,3,17:06:47,8,23.258889,0.0,-1.333275,36.870815,-1.305249,36.822390,Rider_Id_192,16,44,16,44,17,6,16,53,1,1,1,1,12,1
1,Order_No_12736,User_Id_3647,Bike,3,Business,17,5,12:57:35,17,5,12:59:17,17,5,13:20:27,17,5,13:25:37,5,23.258889,0.0,-1.272639,36.794723,-1.277007,36.823907,Rider_Id_868,12,57,12,59,13,25,13,20,1,1,1,1,9,9
2,Order_No_768,User_Id_2154,Bike,3,Business,27,4,11:08:14,27,4,11:25:05,27,4,11:33:20,27,4,11:57:54,5,22.800000,0.0,-1.290894,36.822971,-1.276574,36.851365,Rider_Id_26,11,8,11,25,11,57,11,33,1,1,1,1,7,9
3,Order_No_15332,User_Id_2910,Bike,3,Business,17,1,13:51:35,17,1,13:53:27,17,1,14:02:41,17,1,14:16:52,5,24.500000,0.0,-1.290503,36.809646,-1.303382,36.790658,Rider_Id_685,13,51,13,53,14,16,14,2,1,1,1,1,7,0
4,Order_No_21373,User_Id_1205,Bike,3,Business,11,2,11:30:28,11,2,11:34:45,11,2,11:47:19,11,2,11:56:04,6,24.400000,0.0,-1.281081,36.814423,-1.266467,36.792161,Rider_Id_858,11,30,11,34,11,56,11,47,1,1,1,1,7,7


In [315]:
test.dtypes

order_no                            object
user_id                             object
vehicle_type                        object
platform_type                        int64
personal_or_business              category
placement_day_of_month               int64
placement_weekday_mo_1               int64
placement_time                      object
confirmation_day_of_month            int64
confirmation_weekday_mo_1            int64
confirmation_time                   object
arrival_at_pickup_day_of_month       int64
arrival_at_pickup_weekday_mo_1       int64
arrival_at_pickup_time              object
pickup_day_of_month                  int64
pickup_weekday_mo_1                  int64
pickup_time                         object
distance_km                          int64
temperature                        float64
precipitation_in_millimeters       float64
pickup_lat                         float64
pickup_long                        float64
destination_lat                    float64
destination

In [0]:

train['placement_hourfloat']=train.placement_hour+train.placement_minute/60.0

train['confirmation_hourfloat']=train.confirmation_hour+train.confirmation_minute/60.0

train['pickup_hourfloat']=train.pickup_hour+train.pickup_minute/60.0

train['arrival_hourfloat']=train.arrival_at_pickup_time_hour+train.arrival_at_pickup_time_minute/60.0




train['placement_x']=np.sin(2.*np.pi*train.placement_hourfloat/24.)

train['confirmation_x']=np.sin(2.*np.pi*train.confirmation_hourfloat/24.)

train['arrival_x']=np.sin(2.*np.pi*train.arrival_hourfloat/24.)

train['pickup_x']=np.sin(2.*np.pi*train.pickup_hourfloat/24.)




train['placement_y']=np.cos(2.*np.pi*train.placement_hourfloat/24.)

train['confirmation_y']=np.cos(2.*np.pi*train.confirmation_hourfloat/24.)

train['arrival_y']=np.cos(2.*np.pi*train.arrival_hourfloat/24.)

train['pickup_y']=np.cos(2.*np.pi*train.pickup_hourfloat/24.)





# TEST
test['placement_hourfloat']=test.placement_hour+test.placement_minute/60.0

test['confirmation_hourfloat']=test.confirmation_hour+test.confirmation_minute/60.0

test['pickup_hourfloat']=test.pickup_hour+test.pickup_minute/60.0

test['arrival_hourfloat']=test.arrival_at_pickup_time_hour+test.arrival_at_pickup_time_minute/60.0




test['placement_x']=np.sin(2.*np.pi*test.placement_hourfloat/24.)

test['confirmation_x']=np.sin(2.*np.pi*test.confirmation_hourfloat/24.)

test['arrival_x']=np.sin(2.*np.pi*test.arrival_hourfloat/24.)

test['pickup_x']=np.sin(2.*np.pi*test.pickup_hourfloat/24.)



test['placement_y']=np.cos(2.*np.pi*test.placement_hourfloat/24.)

test['confirmation_y']=np.cos(2.*np.pi*test.confirmation_hourfloat/24.)

test['arrival_y']=np.cos(2.*np.pi*test.arrival_hourfloat/24.)

test['pickup_y']=np.cos(2.*np.pi*test.pickup_hourfloat/24.)



In [317]:
train.columns

Index(['order_no', 'user_id', 'vehicle_type', 'platform_type',
       'personal_or_business', 'placement_day_of_month',
       'placement_weekday_mo_1', 'placement_time', 'confirmation_day_of_month',
       'confirmation_weekday_mo_1', 'confirmation_time',
       'arrival_at_pickup_day_of_month', 'arrival_at_pickup_weekday_mo_1',
       'arrival_at_pickup_time', 'pickup_day_of_month', 'pickup_weekday_mo_1',
       'pickup_time', 'distance_km', 'temperature',
       'precipitation_in_millimeters', 'pickup_lat', 'pickup_long',
       'destination_lat', 'destination_long', 'rider_id',
       'time_from_pickup_to_arrival', 'placement_hour', 'placement_minute',
       'confirmation_hour', 'confirmation_minute', 'pickup_hour',
       'pickup_minute', 'arrival_at_pickup_time_hour',
       'arrival_at_pickup_time_minute', 'placement_ap', 'confirmation_ap',
       'pickup_ap', 'arrival_ap', 'pickup_cluster_label',
       'destination_cluster_label', 'placement_hourfloat',
       'confirmation_h

In [0]:
x = train.drop(['time_from_pickup_to_arrival','order_no','user_id','vehicle_type', 'confirmation_time', 'arrival_at_pickup_time','pickup_time','placement_time','rider_id'],axis=1)
X = pd.get_dummies(x)
y = train["time_from_pickup_to_arrival"]

In [0]:
test1 = test.drop(['user_id','vehicle_type','order_no', 'confirmation_time', 'arrival_at_pickup_time','pickup_time','placement_time','rider_id'],axis=1)
test2 = pd.get_dummies(test1)

In [0]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler,MinMaxScaler

In [321]:
X.shape

(21201, 72)

In [322]:
test2.shape

(7068, 72)

In [0]:
sc =StandardScaler()
X = sc.fit_transform(X)
test = sc.transform(test2)
pca =PCA()
X  =  pca.fit_transform(X)
test = pca.transform(test2)

In [0]:
pca =PCA()
X  =  pca.fit_transform(X)
test = pca.transform(test2)

In [0]:
from sklearn.discriminant_analysis  import LinearDiscriminantAnalysis as LDA

In [0]:
# lda =LDA()
# X  =  lda.fit_transform(X,y)
# test = lda.transform(test2)

In [0]:
clf = lgb.LGBMRegressor(learning_rate= 0.1, min_data_in_leaf= 300, n_estimators= 100, num_leaves= 15, objective='regression', reg_alpha= 0.02)
clf.fit(X,y)
pred = clf.predict(test)

clf_output = pd.DataFrame({"Order_No":sub['Order_No'], 
                           "Time from Pickup to Arrival": pred })
clf_output.to_csv("submission6.csv", index=False)